In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

from keras.models import load_model
from tensorflow.keras.models import load_model

import keras
from keras.preprocessing import image

from PIL import Image

from skimage import transform
from google.colab import files

Using TensorFlow backend.


In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = None)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

--2019-08-31 01:14:43--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  77.0MB/s    in 1.1s    

2019-08-31 01:14:45 (77.0 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



W0831 01:14:45.448911 140275914188672 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [0]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

model.summary()

W0831 01:15:12.596893 140275914188672 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
main = '/content/drive/My Drive/Datasets/dogcat'
savepath = '/content/drive/My Drive/Datasets/dogcat/'

In [0]:
TRAINING_DIR = main + '/training_set'
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=100, class_mode='binary', target_size=(150, 150))

VALIDATION_DIR = main + '/test_set'
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size=100, class_mode='binary',target_size=(150, 150))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [0]:
call = myCallback()

history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 1,
            callbacks = [call])

Epoch 1/100
100/100 [==============================] - 3524s 35s/step - loss: 0.3699 - acc: 0.8781 - val_loss: 0.3023 - val_acc: 0.9336
Epoch 2/100
100/100 [==============================] - 72s 716ms/step - loss: 0.1139 - acc: 0.9589 - val_loss: 0.3857 - val_acc: 0.9306
Epoch 3/100
100/100 [==============================] - 71s 706ms/step - loss: 0.0678 - acc: 0.9796 - val_loss: 0.4800 - val_acc: 0.9252
Epoch 4/100
100/100 [==============================] - 70s 705ms/step - loss: 0.0406 - acc: 0.9900 - val_loss: 0.5987 - val_acc: 0.9262
Epoch 5/100
100/100 [==============================] - 70s 703ms/step - loss: 0.0183 - acc: 0.9955 - val_loss: 0.5215 - val_acc: 0.9314
Epoch 6/100
100/100 [==============================] - 71s 709ms/step - loss: 0.0176 - acc: 0.9965 - val_loss: 0.5950 - val_acc: 0.9304
Epoch 7/100
100/100 [==============================] - 70s 703ms/step - loss: 0.0251 - acc: 0.9965 - val_loss: 1.0990 - val_acc: 0.8988
Epoch 8/100
100/100 [===========================

In [0]:
model.save(savepath + 'petclassification.h5')

In [0]:
import numpy as np
from tensorflow.keras.models import load_model
from keras.preprocessing import image
from google.colab import files

model = load_model('/content/drive/My Drive/Datasets/dogcat/petclassification.h5')

uploaded=files.upload()

for fn in uploaded.keys():
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images)
  if classes[0]>0:
    print(fn + " is a cat")
    
  else:
    print(fn + " is a dog") 

Saving images.jpg to images (2).jpg
images.jpg is a cat
